# PSTA

In [1]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import math
from multiprocessing import  Pool

In [5]:
df = pd.read_csv('../../Semesteroppgave/datasets/dataset/dataset_twitter/tweetsreplies4.tsv', encoding='cp1252', sep="\t", usecols=['timestamp_ms','longitude', 'latitude', 'text'])

## Preprocessing
Get detailed locations and timestamp and tokenize the text.
* Fetch more location info from longitude and latitude using reverse_encoder https://github.com/thampiman/reverse-geocoder
* Convert ms timestamps to datetime object.
* Preprocess text using the preprocessor https://github.com/s/preprocessor and remove stopwords using gensim.

In [6]:
coordinates = list(df[['latitude','longitude']].itertuples(index=False, name=None))
locations = rg.search(coordinates)

Loading formatted geocoded file...


In [7]:
locations_df = pd.json_normalize(locations)[['name', 'admin1', 'admin2', 'cc']]

In [8]:
df = pd.concat([df, locations_df], axis=1)

In [9]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [10]:
df = df.drop(df[df.text == ''].index)
df = df.drop(df[df.text.isna()].index)
df = df.drop(585341+536) # this row makes the function p.clean crash...

In [11]:
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.NUMBER)
def add_features(df):
    try:
        df['text'] = df['text'].apply(p.clean)
    except:
        print(df['text'])
    return df
df = parallelize_dataframe(df, add_features)

In [29]:
# make lower case and remove numbers and white space, must be done after cleaning
df['text']  = df['text'] \
                .str.lower() \
                .str.replace('\d+', '') \
                .str.replace('a{2,}', 'a') \
                .str.replace('b{3,}', 'b') \
                .str.replace('c{3,}', 'c') \
                .str.replace('d{3,}', 'd') \
                .str.replace('e{3,}', 'e') \
                .str.replace('f{3,}', 'f') \
                .str.replace('g{3,}', 'g') \
                .str.replace('h{3,}', 'h') \
                .str.replace('i{2,}', 'i') \
                .str.replace('j{3,}', 'j') \
                .str.replace('k{3,}', 'k') \
                .str.replace('l{3,}', 'l') \
                .str.replace('m{3,}', 'm') \
                .str.replace('n{3,}', 'n') \
                .str.replace('o{3,}', 'o') \
                .str.replace('p{3,}', 'p') \
                .str.replace('q{3,}', 'q') \
                .str.replace('r{3,}', 'r') \
                .str.replace('s{3,}', 's') \
                .str.replace('t{3,}', 't') \
                .str.replace('u{2,}', 'u') \
                .str.replace('v{3,}', 'v') \
                .str.replace('w{3,}', 'w') \
                .str.replace('x{3,}', 'x') \
                .str.replace('y{2,}', 'y') \
                .str.replace('z{3,}', 'z') \
                .str.replace('_', ' ') \
                .str.replace(' rt ', '') \
                .str.replace('#', '') \
                .str.replace('[^\w\s]',' ') \
                .str.replace('\s\s+', ' ')

In [30]:
# remove stopwords, must be done after cleaning and removal of white space
def remove_stopwords_f(df):
    df['text'] = df['text'].apply(remove_stopwords)
    return df
df = parallelize_dataframe(df, remove_stopwords_f)

In [31]:
df = df.drop(df[df.text == ''].index)

In [32]:
df = df.sort_values(by=['timestamp_ms', 'admin1'])

In [33]:
df = df.drop(['admin2'], axis=1)

KeyError: "['admin2'] not found in axis"

In [34]:
df

,timestamp_ms,longitude,latitude,text,name,admin1,cc
0,1443887012127,-3.703508,40.477795,el bara lleva penaltis favor y el madrid slo v...,Tetuan de las Victorias,Madrid,ES
3,1443887056044,-5.923545,37.383117,envidia,Sevilla,Andalusia,ES
7,1443887073903,-66.578926,6.422820,acuerdo pero otra cosa es que ellos lo sepan c...,Puerto Carreno,Vichada,CO
5,1443887110156,-0.352529,39.422484,tu eres retrasado,Alfafar,Valencia,ES
13,1443887259772,20.951529,52.230058,oblaem si herbat,Ochota,Masovian Voivodeship,PL
...,...,...,...,...,...,...,...
707997,1573501662496,29.005222,41.021321,nasl gzel elolu neler neler yapyor zenmemek el...,UEskuedar,Istanbul,TR
849012,1574143147170,32.663877,39.914185,sundun habere sen inandn m zor artlarda altnz ...,Etimesgut,Ankara,TR
463066,1574422333977,-45.917442,-23.061418,bom dia junko,Sao Jose dos Campos,Sao Paulo,BR
392692,1575247560716,-80.476404,43.430240,radio stationi listen everyday turns day every...,Kitchener,Ontario,CA


In [35]:
fmt = '%d\n%.8f\n%.8f\n%s\n%s\n%s\n%s'
np.savetxt(r'datasetAll.txt', df.values, fmt=fmt, delimiter='\r\n')

In [4]:

p.clean("#what is https://vg.no @me FAV00 00 RT _")

'#what is FAV00 RT _'